In [50]:
# !conda install -c conda-forge python-dotenv -y
import numpy as np
import pandas as pd
from deap import base, creator, tools
from functools import reduce
from dotenv import load_dotenv
import finlab
import os
import plotly.express as px
from finlab import data
from finlab.backtest import sim
from finlab.dataframe import FinlabDataFrame
from finlab.portfolio import Portfolio
from finlab.plot import plot_tw_stock_candles
from finlab.data import indicator
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta

# 載入.env檔案中的環境變數

load_dotenv()
# 這裡要替換成自己的FinLab VIP token
token = os.environ.get("FINLAB_TOKEN")
finlab.login(token)



輸入成功!


## ATR波動率的定義
ATR（Average True Range，平均真實波幅）是一種常用的波動率指標，主要用於衡量市場價格的波動程度。  
finlab:https://www.finlab.tw/tw-stock-market-atr/  


#### ATR的計算方法：
1. 首先計算每日的「真實波幅」（True Range, TR），TR是以下三個值中的最大值：
   - 當日最高價減去當日最低價
   - 當日最高價減去前一日收盤價的絕對值
   - 當日最低價減去前一日收盤價的絕對值

2. 然後對一段時間內（通常是14天）的真實波幅取平均值，得到ATR值。

### ATR的主要特點與用途：
- **波動率衡量**：ATR值越大，表示市場波動越劇烈；ATR值越小，表示市場波動較小。
- **不考慮方向**：ATR只衡量波動的大小，不考慮價格移動的方向（上漲或下跌）。
- **資金管理工具**：交易者常用ATR來設定止損位置，例如設定在當前價格的1.5倍或2倍ATR距離。
- **交易系統參數**：可用於自適應調整交易系統的參數，使系統能夠根據市場波動性自動調整。
- **市場狀態判斷**：ATR可以幫助判斷市場是處於高波動（可能是趨勢形成或轉折點）還是低波動（可能是盤整或趨勢減弱）的狀態。

### 在策略開發中，ATR經常被用於：
- 設定合理的止損位置
- 確定持倉規模
- 識別波動率突破
- 作為其他指標的過濾器

如果你在finlab中使用ATR，可以考慮將其納入到你的策略中作為風險管理或交易信號過濾的工具。
使用還原全息股價，計算 ATR/收盤價。   
使用 ATR 的好處在除了最高最低價，還會會考慮 K 棒跳空因素，會比較符合波動真實情況。  
接著再取每日市場前 20% 波動率高的標地，計算市場上波動大的股票當市場指標，排掉波動過小的標地。 最後取指標的5日均線做平滑。

In [35]:
adj_close = data.get('etl:adj_close')
adj_high = data.get('etl:adj_high')
adj_low = data.get('etl:adj_low')

#計算調整後的最高價與最低價之間的差值，即日內價格波動的絕對幅度
amplitude = abs(adj_high - adj_low)/(adj_close.shift()) 

#使用quantile_row(0.8)找出每一行（假設是不同股票）的80%分位數,
#只保留那些振幅大於此80%分位數的值（即篩選出最劇烈波動的20%情況）
#.mean(axis=1)：對這些高波動值按行（每個時間點）取平均
amplitude_ind = amplitude[amplitude > amplitude.quantile_row(0.8)].mean(axis=1).rolling(5).mean()
amplitude_ind['2024-08-12']

np.float64(0.08432219785911625)

### Note

- **高波動(8%以上)是買點**  
超高波動往往出現在大盤急跌末段，許多股票殺低後V轉的時候，最近2次都和疫情的恐慌有關。  
之後的波動率會開始往下掉。 目前該指標約在 0.06 左右，波動率未達過去指數崩跌後的V轉水準，能量還在釋放中，持續有餘震，要小心。  

- **低波動時期(4%以下)是買點**  
股票逐底沈澱的階段，最近一次是今年2月，指數相對低點的時候，市場人氣低迷，沒人買股票。這時若展開牛市，波動率會開始變高。  
指數若上漲一大段後，要注意指數不動、波動率開始降低的時候，可能是人氣退潮的訊號。  
現在的市場 ATR波動率 確實創今年新高，不過和歷史比，目前並沒有特別高，雖然感覺 AI 概念股很旺，但其可能在整體市場的檔數沒想像中多。  

---

### 在股票即將飆漲之前和飆漲期間，波動率通常會有以下幾種典型變化模式：

1. **前兆階段的波動率收縮**:
   - 在大幅上漲前，常常會先經歷一段波動率明顯降低的時期
   - ATR值相對較低，價格波動範圍變小
   - 這種低波動率的壓縮狀態往往是能量積累的表現
   - 技術分析師稱之為「波動率壓縮」(Volatility Squeeze)

2. **突破時的波動率爆發**:
   - 當價格開始突破時，波動率會突然顯著增加
   - ATR值會快速上升
   - 通常伴隨著成交量的明顯放大
   - 這是能量釋放的階段

3. **持續上漲階段的波動特徵**:
   - 如果是健康的上漲趨勢，波動率會維持在較高水平但趨於穩定
   - 漲幅較大的日子ATR增加，盤整日ATR略微下降
   - 波動率通常會呈現方向性，表現為「真實波幅」中的主要貢獻來自最高價與前一日收盤價的差距

4. **末期特徵**:
   - 在上漲接近尾聲時，波動率往往會達到極端高點
   - 日內波動範圍擴大，經常出現大幅高開後回落或V型反轉的走勢
   - 這種極端波動通常是市場情緒過熱的信號

從策略角度來看，可以考慮：
- 使用波動率壓縮作為潛在突破的預警信號
- 結合波動率突然增加和價格突破作為入場信號
- 波動率達到異常高位時提高警惕，考慮獲利了結

需要注意的是，波動率只是眾多指標之一，單獨使用波動率預測股價走向並不可靠，最好結合價格形態、成交量、基本面等多方面因素進行分析。

In [58]:
# 10日的ATR波動率
atr = data.indicator('ATR', adjust_price=True,timeperiod=3)
adj_close = data.get('etl:adj_close')
entry_volatility = atr/adj_close

# 計算20天前的ATR
# 取得今天日期
today = pd.Timestamp.today().normalize()  # normalize()去除時間部分
# 計算10天前
twenty_days_ago = today - timedelta(days=20)
today =today.strftime('%Y-%m-%d')
twenty_days_ago=twenty_days_ago.strftime('%Y-%m-%d')
print(twenty_days_ago)  


entry_volatility['8096'].loc[:twenty_days_ago]
print(entry_volatility.loc[:twenty_days_ago])


2025-02-26
                0015      0050      0051      0052      0053  0054      0055  \
date                                                                           
2007-04-23       NaN       NaN       NaN       NaN       NaN   NaN       NaN   
2007-04-24       NaN       NaN       NaN       NaN       NaN   NaN       NaN   
2007-04-25       NaN       NaN       NaN       NaN       NaN   NaN       NaN   
2007-04-26  0.005561  0.009243  0.007419  0.008117       NaN   NaN       NaN   
2007-04-27  0.005119  0.009662  0.007912  0.007176       NaN   NaN       NaN   
...              ...       ...       ...       ...       ...   ...       ...   
2025-02-20       NaN  0.009563  0.006712  0.010373  0.011452   NaN  0.005981   
2025-02-21       NaN  0.009782  0.008174  0.009977  0.010756   NaN  0.005014   
2025-02-24       NaN  0.010742  0.007524  0.011213  0.011324   NaN  0.004961   
2025-02-25       NaN  0.013452  0.008169  0.014680  0.013618   NaN  0.005731   
2025-02-26       NaN  0.01371

## 找波動率因子
1. 波動大後變波動小  
ex : 長日前的波動大(漲跌幅劇烈) , 但近日波動小(盤整休息)

2. 波動緩慢增加變大
ex : 起漲時,前面波動比較小,整理中,上漲幾個紅K波動就變大

In [41]:
# 10日的ATR波動率
atr = data.indicator('ATR', adjust_price=True,timeperiod=10)
adj_close = data.get('etl:adj_close')
entry_volatility = atr/adj_close
print(entry_volatility)

            0015      0050      0051      0052      0053  0054      0055  \
date                                                                       
2007-04-23   NaN       NaN       NaN       NaN       NaN   NaN       NaN   
2007-04-24   NaN       NaN       NaN       NaN       NaN   NaN       NaN   
2007-04-25   NaN       NaN       NaN       NaN       NaN   NaN       NaN   
2007-04-26   NaN       NaN       NaN       NaN       NaN   NaN       NaN   
2007-04-27   NaN       NaN       NaN       NaN       NaN   NaN       NaN   
...          ...       ...       ...       ...       ...   ...       ...   
2025-03-12   NaN  0.016442  0.012350  0.018689  0.016283   NaN  0.008071   
2025-03-13   NaN  0.017588  0.013394  0.019788  0.017043   NaN  0.008225   
2025-03-14   NaN  0.016768  0.012605  0.019073  0.016707   NaN  0.007891   
2025-03-17   NaN  0.016610  0.012508  0.018921  0.016002   NaN  0.008548   
2025-03-18   NaN  0.015873  0.012115  0.018088  0.015439   NaN  0.008669   

           

### 程式碼
https://colab.research.google.com/drive/1qzV59q_ydeUhlPiPeGGz-WsHrkpZd1rG?usp=sharing#scrollTo=fZP5bhO93Pi4

In [32]:
# =====發行量加權股價報酬指數=======
benchmark_return = data.get('benchmark_return:發行量加權股價報酬指數')
benchmark_return = benchmark_return.loc[amplitude_ind.index]
# =====波動率=======
entry_volatility_ind = entry_volatility[entry_volatility > entry_volatility.quantile_row(0.8)].mean(axis=1).rolling(5).mean()
entry_volatility_ind

# ======================= 畫圖 ====================
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=benchmark_return.index, y=benchmark_return['發行量加權股價報酬指數'], name="大盤加權報酬指數"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=entry_volatility_ind.index, y=entry_volatility_ind.values, name="市場ATR波動率指標"),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title={
        'text': "台股市場ATR波動率指標 V.S 大盤加權報酬指數",
        'x': 0.49,
        'y': 0.9,
        'xanchor': 'center',
        'yanchor': 'top'},
)



# Set y-axes titles
fig.update_yaxes(title_text="<b>大盤加權報酬指數</b>", secondary_y=False, showgrid=False)
fig.update_yaxes(title_text="<b>ATR波動率指標</b>", secondary_y=True)
# Set x-axis title
fig.update_xaxes(title_text="date",
                 rangeselector=dict(
                     buttons=list([
                         dict(count=1,
                              label="1y",
                              step="year",
                              stepmode="backward"),
                         dict(count=3,
                              label="3y",
                              step="year",
                              stepmode="backward"),
                         dict(count=5,
                              label="5y",
                              step="year",
                              stepmode="backward"),
                         dict(count=10,
                              label="10y",
                              step="year",
                              stepmode="backward"),
                         dict(step="all")
                     ])
                 ),
                 rangeslider=dict(
                     visible=True
                 ),
                 type="date")

fig.show()